In [21]:
import pandas as pd
from database.scrape import get_all_stats
from modeling.predict import Predict
from st_helpers.table_functions import create_streamlit_table, create_summary_stats
from sklearn.ensemble import RandomForestRegressor
pd.set_option('display.max_columns', 70)

In [22]:
df_keys = pd.read_csv(r'database\PLAYER_CODE_KEYS.csv')

In [23]:
df_select = df_keys.loc[df_keys['Player'].isin(['Goff, Jared', 'Prescott, Dak'])]

In [24]:
stats = [get_all_stats(i) for i in df_select['Player Code']]
df_raw_stats = pd.concat(stats)

P/PresDa01 done.
G/GoffJa00 done.


In [25]:
st_table = create_streamlit_table(data=df_raw_stats, PPR_adj=1, pass_td_adj=4)

In [26]:
player_pred = [
Predict(
    df=st_table, player=i, steps=1, regressor=RandomForestRegressor, lags=9
).predict_season() for i in list(set(st_table['PLAYER']))
]

c:\Users\012356.CORPORATE\Fantasy_Football_Machine_Learning\modeling\predict.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.date_range(
c:\Users\012356.CORPORATE\Fantasy_Football_Machine_Learning\modeling\predict.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.to_datetime(
c:\Users\012356.CORPORATE\Fantasy_Football_Machine_Learning\modeling\predict.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [27]:
clean_data = [row for row in player_pred if row is not None]
df_preds = pd.DataFrame(clean_data, columns=['PLAYER', 'Next Game(s) Projection'])

,PLAYER,Next Game(s) Projection
0,Jared Goff,18.2136
1,Dak Prescott,17.8112


In [28]:
st_table.reset_index(inplace=True)

In [29]:
create_summary_stats(data=st_table, combine_with=df_preds)

,PLAYER,Next Game(s) Projection,Rolling_3_Avg,Rolling_5_Avg,Rolling_8_Avg,Rolling_8_Stddev
0,Jared Goff,18.2136,15.653333,15.216,16.745,6.543994
1,Dak Prescott,17.8112,18.513333,19.692,19.090,6.702750
